In [20]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid,normalize_0_1
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang, eq_to_seq
import gym, ray
from ray.rllib.agents import ppo
import numpy as np

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
ray.init(ignore_reinit_error=True)


2021-07-07 17:24:12,176	INFO worker.py:745 -- Calling ray.init() again after it has already been called.


In [22]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

In [23]:
input_lang.n_words

4123

In [24]:
# def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:
#     # print("output_sequence ", output_sequence)
#     # print("target_sequence ", output_sequence)
#     magnitude: float = 0.0
#     for x, y in zip(output_sequence, target_sequence):
#         magnitude += abs(x - y)
#     # magnitude /= len(norm_target_seq)
#     return 10 - magnitude


In [32]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:

    if len(output_sequence) != len(target_sequence):
        raise AssertionError("sequence size don't match: " + ','.join(str(e) for e in output_sequence)
 + " | " + ','.join(str(e) for e in target_sequence))
    print("working prediction")

    magnitude: float = 0.0

    combined_seq = np.vstack([output_sequence, target_sequence]) 
    norm_comb_seq = normalize_0_1(combined_seq)

    norm_output_seq = norm_comb_seq[0]
    norm_target_seq = norm_comb_seq[1]

    
    for x, y in zip(norm_target_seq, norm_output_seq):
        magnitude += abs(x - y)#**2

    # magnitude /= len(norm_target_seq)

    return 10 - (magnitude * 100)



In [33]:
compare_sequences([1,2, 3, 100], [1,2,3, 100])

working prediction


10.0

In [34]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return -MAX_PENALTY_MAGNITUDE

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return -MAX_PENALTY_MAGNITUDE

    return compare_sequences(output_sequence, int_seq)

In [35]:
len(train[0][0])

8

In [36]:
env = IntegerSequenceEnv({"int_sequence": train[0][0], "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "evaluate": evaluate_candidate_eq})

In [41]:
trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
    "env_config": {
        "int_sequence": train[0][0],
        "output_length": 9,
        "input_lang": input_lang,
        "output_lang": output_lang,
        "evaluate": evaluate_candidate_eq
    },
    "num_envs_per_worker": 1,

})

Exception: Unknown config parameter `config` 

In [42]:
trainer.train()

StopIteration: 